## **Koala7B 8bit + HuggingFaceEmbedding + Llama Index**

Code released by: autratec
This github version were developed from v2

The POC project below enable resercher to setup large language model locally or leverage google colab free environment, wiht vector embedding technology from HaggingFace to do provide an accurate response based on local content through indexing. 

Colab resouce usage:  RAM: 5.4G. GPU8.9G

Pls create a folder callled data and get yoru raw data (csv) in that folder and index.json will be created under root path. Enjoy your test. 

Putting pipeline outside of class to reduce GPU usage. 

Here are the reference of codes being used in this notebook: 

https://colab.research.google.com/drive/10QPfcDt39uGciEDqdYBAbPBNZQDoC99O?usp=sharing

https://discord.com/channels/1059199217496772688/1090945925129707570



In [1]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate
!pip -q install langchain transformers sentence_transformers llama-index

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 28.

In [3]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch
from llama_index import SimpleDirectoryReader, GPTSimpleVectorIndex, PromptHelper, LLMPredictor, ServiceContext, LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms.base import LLM

In [4]:
tokenizer = LlamaTokenizer.from_pretrained("samwit/koala-7b")
model = LlamaForCausalLM.from_pretrained("samwit/koala-7b",load_in_8bit=True,device_map='auto',)
pipeline = pipeline("text-generation",model=model, tokenizer=tokenizer, max_length=512,temperature=0.7,top_p=0.95,repetition_penalty=1.15)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--listen_host=172.28.0.12 --target_host=172.28

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [5]:
class customLLM(LLM):
    def _call(self, prompt, stop=None):
        res = pipeline(prompt)
        prompt_length = len(prompt)
        return res[0]["generated_text"][prompt_length:] 
    def _identifying_params(self):
        return {"name_of_model": "koala-7b"}
    def _llm_type(self):
        return "custom"

Simple test to ensure LLM is working.

In [6]:
print(customLLM()._call("Tell me somthing about New York City."))



A: Sure, I can tell you something about New York City! It's a city located in the state of New York on the East Coast of the United States. It is known for its iconic landmarks such as the Empire State Building and Times Square, as well as its diverse culture and vibrant energy. The city has a rich history and is home to many world-class museums, galleries, and restaurants. If you ever have the chance to visit New York City, be sure to check out some of these attractions and experience all that this amazing city has to offer!


In [7]:
max_input_size = 512
num_output = 200
max_chunk_overlap = 20
chunk_size_limit = 200

llm_predictor = LLMPredictor(llm=customLLM())

In [8]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())

In [9]:
prompt_helper = PromptHelper(max_input_size, num_output,max_chunk_overlap,chunk_size_limit=chunk_size_limit)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model, prompt_helper=prompt_helper, chunk_size_limit = chunk_size_limit) 

Create a folder called "data" and load your csv file for indexing. 

In [10]:
documents = SimpleDirectoryReader('./data').load_data()
index = GPTSimpleVectorIndex.from_documents(documents,service_context=service_context)
index.save_to_disk('index.json')

In [19]:
query_text = "How to pick right resouce to join a project team ?"
response = index.query(query_text,response_mode="compact",service_context=service_context, similarity_top_k=1)
print(response)


Answer: To pick the right resource to join a project team, consider the following factors:

1.   Technical skills: The technical skills required for the project should be considered when selecting resources. This includes their proficiency in tools and technologies used in the project.
2.   Experience: The experience level of the resource should also be taken into account when selecting them for the project team. Resources with more experience may have better problem-solving skills and can handle complex tasks more effectively than those with less experience.
3.   Availability: The availability of the resource should be considered when selecting them for the project team. If the resource is not available during the project timeline, it will delay the project's progress.
4.   Communication skills: Good communication skills are essential for successful collaboration within the project team. It is important to select resources who can communicate effectively with other team members and s